In [ ]:
!wget -O Gaze360.tar "http://gaze360.csail.mit.edu/dataset.php?email=a.magadirajeev@student.tudelft.nl&hash=2a79ea27c279e471f4d180b08d62b00a"

--2023-03-21 16:37:43--  http://gaze360.csail.mit.edu/dataset.php?email=a.magadirajeev@student.tudelft.nl&hash=2a79ea27c279e471f4d180b08d62b00a
Resolving gaze360.csail.mit.edu (gaze360.csail.mit.edu)... 128.30.2.44
Connecting to gaze360.csail.mit.edu (gaze360.csail.mit.edu)|128.30.2.44|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30548756480 (28G) [application/x-tar]
Saving to: ‘Gaze360.tar’

Gaze360.tar         100%[===================>]  28.45G  6.35MB/s    in 90m 47s 

2023-03-21 18:08:31 (5.35 MB/s) - ‘Gaze360.tar’ saved [30548756480/30548756480]



In [ ]:
!tar -xvf  'Gaze360.tar' -C '/content/Gaze360data'

Streaming output truncated to the last 5000 lines.
imgs/rec_041/body/000457/000333.jpg
imgs/rec_041/body/000457/000494.jpg
imgs/rec_041/body/000457/000445.jpg
imgs/rec_041/body/000457/000024.jpg
imgs/rec_041/body/000457/000206.jpg
imgs/rec_041/body/000457/000570.jpg
imgs/rec_041/body/000457/000111.jpg
imgs/rec_041/body/000457/000049.jpg
imgs/rec_041/body/000457/000098.jpg
imgs/rec_041/body/000457/000035.jpg
imgs/rec_041/body/000457/000322.jpg
imgs/rec_041/body/000457/000453.jpg
imgs/rec_041/body/000457/000032.jpg
imgs/rec_041/body/000457/000325.jpg
imgs/rec_041/body/000457/000482.jpg
imgs/rec_041/body/000457/000201.jpg
imgs/rec_041/body/000457/000577.jpg
imgs/rec_041/body/000457/000116.jpg
imgs/rec_041/body/000457/000388.jpg
imgs/rec_041/body/000457/000273.jpg
imgs/rec_041/body/000457/000164.jpg
imgs/rec_041/body/000457/000579.jpg
imgs/rec_041/body/000457/000118.jpg
imgs/rec_041/body/000457/000421.jpg
imgs/rec_041/body/000457/000040.jpg
imgs/rec_041/body/000457/000091.jpg
imgs/rec_041/

# Data Pre processing

In [ ]:
import numpy as np
import scipy.io as sio
import cv2 
import os
import sys
sys.path.append("../core/")
import data_processing_core as dpc

root = "/content/Gaze360data/"
out_root = "/content/Gaze360data_preprocessed"



In [ ]:
def ImageProcessing_Gaze360():
    msg = sio.loadmat(os.path.join(root, "metadata.mat"))
    
    recordings = msg["recordings"]
    gazes = msg["gaze_dir"]
    head_bbox = msg["person_head_bbox"]
    face_bbox = msg["person_face_bbox"]
    lefteye_bbox = msg["person_eye_left_bbox"]
    righteye_bbox = msg["person_eye_right_bbox"]
    splits = msg["splits"]

    split_index = msg["split"]
    recording_index = msg["recording"]
    person_index = msg["person_identity"]
    frame_index = msg["frame"]
  
    total_num = recording_index.shape[1]
    outfiles = []
    

    # Build folders for saving image and label.
    if not os.path.exists(os.path.join(out_root, "Label")):
        os.makedirs(os.path.join(out_root, "Label"))
        
    for i in range(4):
        if not os.path.exists(os.path.join(out_root, "Image", splits[0, i][0])):
            os.makedirs(os.path.join(out_root, "Image", splits[0, i][0], "Left"))
            os.makedirs(os.path.join(out_root, "Image", splits[0, i][0], "Right"))
            os.makedirs(os.path.join(out_root, "Image", splits[0, i][0], "Face"))

        outfiles.append(open(os.path.join(out_root, "Label", f"{splits[0, i][0]}.label"), 'w')) 
        outfiles[i].write("Face Left Right Origin 3DGaze 2DGaze\n")

    # process each image
    for i in range(total_num):
        im_path = os.path.join(root, "imgs",
            recordings[0, recording_index[0, i]][0],
            "head", '%06d' % person_index[0, i],
            '%06d.jpg' % frame_index[0, i]
            )
        progressbar = "".join(["\033[41m%s\033[0m" % '   '] * int(i/total_num * 20))
        progressbar = "\r" + progressbar + f" {i}|{total_num}"
        print(progressbar, end = "", flush=True)
        if (face_bbox[i] == np.array([-1, -1, -1, -1])).all():
            continue

        category = splits[0, split_index[0, i]][0]
        gaze = gazes[i]

        img = cv2.imread(im_path)
        face = CropFaceImg(img, head_bbox[i], face_bbox[i])
        lefteye = CropEyeImg(img, head_bbox[i], lefteye_bbox[i])
        righteye = CropEyeImg(img, head_bbox[i], righteye_bbox[i]) 
        
        cv2.imwrite(os.path.join(out_root, "Image", category, "Face", f"{i+1}.jpg"), face)
        cv2.imwrite(os.path.join(out_root, "Image", category, "Left", f"{i+1}.jpg"), lefteye)
        cv2.imwrite(os.path.join(out_root, "Image", category, "Right", f"{i+1}.jpg"), righteye)

        gaze2d = GazeTo2d(gaze) 

        save_name_face = os.path.join(category, "Face", f"{i+1}.jpg")
        save_name_left = os.path.join(category, "Left", f"{i+1}.jpg")
        save_name_right = os.path.join(category, "Right", f"{i+1}.jpg")

        save_origin = os.path.join(recordings[0, recording_index[0, i]][0],
            "head", "%06d" % person_index[0, i], "%06d.jpg"% frame_index[0, i])

        save_gaze = ",".join(gaze.astype("str"))
        save_gaze2d = ",".join(gaze2d.astype("str"))

       	
        save_str = " ".join([save_name_face, save_name_left, save_name_right, save_origin, save_gaze, save_gaze2d])
        outfiles[split_index[0, i]].write(save_str + "\n")

    for i in outfiles:
        i.close()
    #

In [ ]:
def GazeTo2d(gaze):
  yaw = np.arctan2(gaze[0], -gaze[2])
  pitch = np.arcsin(gaze[1])
  return np.array([yaw, pitch])

In [ ]:
def CropFaceImg(img, head_bbox, cropped_bbox):
    bbox =np.array([ (cropped_bbox[0] - head_bbox[0])/head_bbox[2],
              (cropped_bbox[1] - head_bbox[1])/head_bbox[3],
              cropped_bbox[2] / head_bbox[2],
              cropped_bbox[3] / head_bbox[3]])

    size = np.array([img.shape[1], img.shape[0]])

    bbox_pixel = np.concatenate([bbox[:2] * size, bbox[2:] * size]).astype("int")

    # Find the image center and crop head images with length = max(weight, height)
    center = np.array([bbox_pixel[0]+bbox_pixel[2]//2, bbox_pixel[1]+bbox_pixel[3]//2])

    length = int(max(bbox_pixel[2], bbox_pixel[3])/2) 

    center[0] = max(center[0], length)
    center[1] = max(center[1], length)

    result = img[(center[1] - length) : (center[1] + length),
                (center[0] - length) : (center[0] + length)] 

    result = cv2.resize(result, (224, 224))
    return result

In [ ]:
def CropEyeImg(img, head_bbox, cropped_bbox):
    bbox =np.array([ (cropped_bbox[0] - head_bbox[0])/head_bbox[2],
              (cropped_bbox[1] - head_bbox[1])/head_bbox[3],
              cropped_bbox[2] / head_bbox[2],
              cropped_bbox[3] / head_bbox[3]])

    size = np.array([img.shape[1], img.shape[0]])

    bbox_pixel = np.concatenate([bbox[:2] * size, bbox[2:] * size]).astype("int")

    center = np.array([bbox_pixel[0]+bbox_pixel[2]//2, bbox_pixel[1]+bbox_pixel[3]//2])
    height = bbox_pixel[3]/36
    weight = bbox_pixel[2]/60
    ratio = max(height, weight) 

    size = np.array([ratio*30, ratio*18]).astype("int")

    center[0] = max(center[0], size[0])
    center[1] = max(center[1], size[1])


    result = img[(center[1] - size[1]): (center[1] + size[1]),
                (center[0] - size[0]): (center[0] + size[0])]

    result = cv2.resize(result, (60, 36)) 
    return result


In [ ]:
if __name__ == "__main__":
    ImageProcessing_Gaze360()

                                                          197587|197588

In [ ]:
#commenthere

In [ ]:
!python3 /content/GazeTR/model.py

In [ ]:
!python3 /content/GazeTR/resnet.py

In [ ]:
import sys
sys.path.insert(0, '/content/GazeTR')
from model import Model
import torch

In [ ]:
!python /content/GazeTR/trainer/leave.py -s /content/GazeTR/config/train/config_gaze360.yaml -p 0

=====================>> (Begin) Training params << =======================
{
    "params": {
        "batch_size": 512,
        "epoch": 50,
        "lr": 0.0005,
        "decay": 0.5,
        "decay_step": 20,
        "warmup": 5
    },
    "save": {
        "metapath": "/content/Transformer/ComTrans/exp/Transformer-6layers",
        "folder": "gaze360",
        "model_name": "trans6",
        "step": 10
    },
    "data": {
        "image": "/content/Gaze360data_preprocessed/Image",
        "label": "/content/Gaze360data_preprocessed/Label",
        "header": true,
        "name": "gaze360",
        "isFolder": false
    },
    "pretrain": {
        "enable": true,
        "path": "/content/Transformer/ComTrans/exp/Transformer-6layers/gaze360/checkpoint/GazeTR-H-ETH.pt",
        "device": 0
    },
    "device": 0,
    "reader": "reader",
    "person": 0
}
=====================>> (End) Traning params << =======================
===> Read data <===
-- [Read Data]: Source: ['/content/Gaz

In [ ]:
!pip install git+https://github.com/ildoonet/pytorch-gradual-warmup-lr.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ildoonet/pytorch-gradual-warmup-lr.git to /tmp/pip-req-build-xjrgj7mv
  Running command git clone --filter=blob:none --quiet https://github.com/ildoonet/pytorch-gradual-warmup-lr.git /tmp/pip-req-build-xjrgj7mv
  Resolved https://github.com/ildoonet/pytorch-gradual-warmup-lr.git to commit 6b5e8953a80aef5b324104dc0c2e9b8c34d622bd
  Preparing metadata (setup.py) ... done


In [1]:
!python /content/GazeTR/tester/leave.py -s /content/GazeTR/config/train/config_gaze360.yaml -t /content/GazeTR/config/test/config_gaze360.yaml -p 0

=======================>(Begin) Config of training<======================
{
    "train": {
        "params": {
            "batch_size": 512,
            "epoch": 50,
            "lr": 0.0005,
            "decay": 0.5,
            "decay_step": 20,
            "warmup": 5
        },
        "save": {
            "metapath": "/content/Transformer/ComTrans/exp/Transformer-6layers",
            "folder": "gaze360",
            "model_name": "trans6",
            "step": 10
        },
        "data": {
            "image": "/content/Gaze360data_preprocessed/Image",
            "label": "/content/Gaze360data_preprocessed/Label",
            "header": true,
            "name": "gaze360",
            "isFolder": false
        },
        "pretrain": {
            "enable": true,
            "path": "/content/Transformer/ComTrans/exp/Transformer-6layers/gaze360/checkpoint/GazeTR-H-ETH.pt",
            "device": 0
        },
        "device": 0,
        "reader": "reader"
    }
}
===============

In [ ]:
!python /content/GazeTR/tester/leave.py -s /content/GazeTR/config/train/config_gaze360.yaml -t /content/GazeTR/config/test/config_gaze360.yaml -p 0

=======================>(Begin) Config of training<======================
{
    "train": {
        "params": {
            "batch_size": 512,
            "epoch": 50,
            "lr": 0.0005,
            "decay": 0.5,
            "decay_step": 20,
            "warmup": 5
        },
        "save": {
            "metapath": "/content/Transformer/ComTrans/exp/Transformer-6layers",
            "folder": "gaze360",
            "model_name": "trans6",
            "step": 10
        },
        "data": {
            "image": "/content/Gaze360data_preprocessed/Image",
            "label": "/content/Gaze360data_preprocessed/Label",
            "header": true,
            "name": "gaze360",
            "isFolder": false
        },
        "pretrain": {
            "enable": true,
            "path": "/content/Transformer/ComTrans/exp/Transformer-6layers/gaze360/checkpoint/GazeTR-H-ETH.pt",
            "device": 0
        },
        "device": 0,
        "reader": "reader"
    }
}
===============